In [1]:
import nengo
import matplotlib.pyplot as plt
import numpy
import gym
import math
%matplotlib inline

# Network details

## Goal Directed/Model-Based

1. LatPFC Maintains a repository representing the environment. Repository of all states and actions.
   Receives cues from the environment and maintains it
   Receives SPE from preSMA and/or pIPS
   
2. LatOFC receives the repository and State Prediction Error from LatPFC and maintains a map of the environment. Something like an internal environment. Updates trainsition probabilities by SPE and updates reward values from dopamine errors.

3. mOFC uses the internal environment from map from LatOFC to do state space search. Receives dopamine errors and passes it to LatOFC to update the internal environment. Passes expected value of the next state after DFS to vmPFC.

4. vmPFC maintains the expected value and passes it to the striatum. Updates expected value for that state from dopamne errors from the striatum and passes it to mOFC.

5. DMS represents expected value. Modulated by Dopamine. Passes it back to vmPFC

6. preSMA/pIPS compare expected state and actual and generates SPE. Passes SPE to latPFC.

## Habitual/Model-Free

1. latPFC maintains the repository of states and passes cue and the repository to dlPFC.

2. dlPFC maintains q_values. Constructs them using the repository from latPFC and dopamine errors from DLS. Uses the cue to decide which value to pass to the DLS.

3. DLS represents expected q_value. Modulated by Dopamine. Passes it back to dlPFC

In [31]:
env = gym.make("FrozenLake-v0")
state = env.reset()

[2017-09-04 12:54:04,768] Making new env: FrozenLake-v0


In [103]:
def to_discrete(vector):
    discrete = vector[0]*4 + vector[1]
    return discrete

def to_vector(discrete):
    vector = numpy.zeros((1, 2))
    vector[1] = discrete%4
    vector[0] = int(discrete/4)
    return vector


In [48]:
class latPFC:
    
    def __init__(self):
        self.state_repo = list()
        self.action_repo = list()
        
    def build_state_repo(self, env):
        for i in range(env.observation_space.n/4):
            for j in range(env.observation_space.n/4):
                self.state_repo.append([i, j])
        self.state_repo = numpy.array(self.state_repo)

    def bulid_action_repo(self, env):
        for i in range(env.action_space.n/4):
            for j in range(env.action_space.n/4):
                self.action_repo.append([i, j])
        self.action_repo = numpy.array(self.action_repo)
    
    def get_cue(self, cue):
        self.state = env.observation_space.sample()
    
    def calc_spe(self, exp_state, actual_state):
        if exp_state == actual_state:
            self.spe = 0
        else:
            self.spe = 1
        return spe
    

In [262]:
#spe is the state prediction error. 1 or 0. 1 is no error
#states is matrix of value for each state
#actions is actions in order "left, down, right, up"

class latOFC:
    def __init__(self, state_repo, action_repo, spe):
        self.spe = spe
        self.spe_sum = 0
        self.count = 0
        self.states = numpy.zeros((len(state_repo)/4, len(state_repo)/4))
        self.states[0][1] = 1
        self.actions = numpy.array([[0, -1], [1, 0], [0, 1], [-1, 0]])

        
    def update_transition_prob(self, spe):
        self.count += 1
        print "count", self.count
        self.spe_sum += spe
        self.transition_prob = float(self.spe_sum)/self.count
        print 
        
        
    def transition(self, state, action, transition_prob):
        if numpy.random.rand() < transition_prob:
            if env.observation_space.contains(to_discrete(numpy.add(state,action))):
                new_state = numpy.add(state, action)
            else:
                new_state = state
        else:
            rand_action = self.actions[env.action_space.sample()]
            if env.observation_space.contains(to_discrete(numpy.add(state,rand_action))):
                new_state = numpy.add(state, rand_action)
            else:
                new_state = state
        return new_state                            

       

In [272]:
class mOFC:
    #transition_prob, states and actions received from latPFC
    def __init__(self, transition_prob, states, actions):
        self.transition_prob = transition_prob
        self.states = states
        self.actions = actions
    
    def dls(self, curr_state, curr_depth, max_depth):
        if curr_depth == max_depth:
            return states(curr_state)
        else:
            for i in self.actions:
                if env.observation_space.contains(to_discrete(numpy.add(curr_state, i))):
                    next_state = numpy.add(curr_state, i)
                    self.states[curr_state] = self.states[next_state] + 0.9*self.dls(self, next_state, curr_depth+1, max_depth)
        

In [264]:
a = latPFC()

In [265]:
a.build_state_repo(env)
a.bulid_action_repo(env)

In [268]:
b = latOFC(a.state_repo, a.action_repo, 0)

In [270]:
c = mOFC(1.0, b.states, b.actions)

States [[ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [273]:
c.dls(numpy.array([0, 0]), 0, 2)

NameError: global name 'dls' is not defined